In [ ]:
def greedy_coloring(graph):
    result = {}
    for task in sorted(graph.keys()):

        adjacent_colors = {result[neighbor] for neighbor in graph[task] if neighbor in result}

        color = 1
        while color in adjacent_colors:
            color += 1
        result[task] = color

    return result


graph = {
    'T1': ['T2', 'T3', 'T5'],
    'T2': ['T1', 'T3', 'T4'],
    'T3': ['T1', 'T2', 'T5'],
    'T4': ['T2', 'T5'],
    'T5': ['T3', 'T4', 'T6'],
    'T6': ['T5', 'T5']
}


coloring = greedy_coloring(graph)

for task in sorted(coloring):
    print(f"{task} -> Time Slot {coloring[task]}")

In [ ]:
    def _calculate_errors(self, X, y, node):
        """Calculate classification errors for a subtree"""
        if node.is_leaf():
            predicted = np.full(len(y), node.value)
            return sum(predicted != y)

        errors = 0
        if node.threshold is not None:  # Numerical split
            left_idxs = X[:, node.feature] <= node.threshold
            right_idxs = X[:, node.feature] > node.threshold
            errors += self._calculate_errors(X[left_idxs], y[left_idxs], node.children["left"])
            errors += self._calculate_errors(X[right_idxs], y[right_idxs], node.children["right"])
        else:  # Categorical split
            for value, child in node.children.items():
                idxs = X[:, node.feature] == value
                errors += self._calculate_errors(X[idxs], y[idxs], child)

        return errors

    def predict(self, X):
        """Predict class labels for samples in X"""
        if isinstance(X, pd.DataFrame):
            X = X.values
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        """Traverse the tree to make a prediction for a single sample"""
        if node.is_leaf():
            return node.value

        if node.threshold is not None:  # Numerical feature
            if x[node.feature] <= node.threshold:
                return self._traverse_tree(x, node.children["left"])
            else:
                return self._traverse_tree(x, node.children["right"])
        else:  # Categorical feature
            value = x[node.feature]
            if value in node.children:
                return self._traverse_tree(x, node.children[value])
            else:
                # If value not seen during training, return most common class
                return self._most_common_label_in_subtree(node)

    def _most_common_label_in_subtree(self, node):
        """Find the most common label in a subtree"""
        if node.is_leaf():
            return node.value

        labels = []
        for child in node.children.values():
            labels.append(self._most_common_label_in_subtree(child))

        counter = Counter(labels)
        return counter.most_common(1)[0][0]